In [1]:
#Initial environement of computations


import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests

from kanon.units import Sexagesimal, Historical, IntegerAndSexagesimal, Temporal
from kanon.calendars import*

Cal=julian_ad = Calendar.registry["Julian A.D."]

number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

# Historical target table
Hist_table=HTable.read(585, with_units=False)
Hist_table


"Years, conjonction",Entries
IntegerAndSexagesimal,Temporal
482495 ;,"28 ; 18,02,06"
491261 ;,"3 ; 19,21,09"
500027 ;,"1 ; 09,24,15"
508793 ;,"12 ; 23,27,21"
517559 ;,"17 ; 13,30,26"
526325 ;,"22 ; 03,33,32"
535091 ;,"26 ; 17,36,38"
543857 ;,"1 ; 18,55,41"
552623 ;,"6 ; 08,58,46"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(13):
    Test_recomp[i][1]=Temporal('0;0,0,0')
    Delta_recomp[i][1]=Temporal('0;0,0,0')


In [11]:
# mean motion parameter from the Parisian Alfonsine tables
daily_mean=Historical(Sexagesimal("12;11,26,41,37,51,50,39"),7)
radice_elong=Historical(Sexagesimal("3,24;25,49,46,11"),4)
delta_Paris_Toledo=Temporal("0;0,48")
REVO=Historical(Sexagesimal("6,0;0,0"),7)
epoch= Date(Cal, (1, 1, 1))

for i in range(13):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=6):

        #conversion of date to DN from calendar epoch
        days=int(Hist_table[i][0])-365

        #computation of the elongation at date
        elong=(days*daily_mean+radice_elong)%REVO

        #computation of time to conjonction including delta between Paris and Toledo
        time=(REVO-elong)/daily_mean
        time=Temporal(time,7)+delta_Paris_Toledo
        
        
        #filling the Test and Delta tables with the final precision
        with set_precision(tmode=TruncatureMode.ROUND, pmode=3):
            Test_recomp[i][1]=1*time
            Delta_recomp[i][1]=24*60*60*(Hist_table[i][1]-Test_recomp[i][1])
Test_recomp

"Years, conjonction",Entries
IntegerAndSexagesimal,Temporal
482495 ;,"28 ; 18,02,06"
491261 ;,"3 ; 19,21,09"
500027 ;,"8 ; 09,24,15"
508793 ;,"12 ; 23,27,21"
517559 ;,"17 ; 13,30,26"
526325 ;,"22 ; 03,33,32"
535091 ;,"26 ; 17,36,38"
543857 ;,"1 ; 18,55,41"
552623 ;,"6 ; 08,58,46"


In [12]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(-46246.153846153844, nan)

In [13]:
B.astype(int).style.background_gradient(axis=None)

,Entries
"Years, conjonction",
482495 ;,0
491261 ;,0
500027 ;,-604800
508793 ;,0
517559 ;,0
526325 ;,0
535091 ;,0
543857 ;,0
552623 ;,0


In [7]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Mean syzygy date years conjonction.csv')
B.to_csv('Mean syzygy date years conjonction.zip', index=False, compression=compression_opts)